<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold;">  Spotify songs </div>

Author: Pierre Mulliez <br>
Last edited: 17/02/2021

<div ><img src="CSS_files/spot.png" width=600 height=400  align="center"></div>




## Summary 
**About this analysis** 
<br>
We would like analyse songs on the spotify platform
<br>
<br>
**Why this analysis**
<br>
Assist parental control upon songs, what songs to play to dance 
<br>
<br>
**How** 
<ul>
<li> Exploring datasets as a panda dataframe </li>
<li> Cleaning and Standardizing the dataset along the way</li>
    <ul>
        <li> Duplicates values</li>
        <li> Entry errors</li>
    </ul>
<li> Party Playlist building  </li>
    <ul>
        <li> Analyse the singer </li>
        <li> Analyse the song </li>
        <li> Analyse the different generations </li>
    </ul>
<li> Note:   </li>
    <ul>
        <li>No machine learning algorithm was used</li>
         <li>Only the following : Popularity, name, artists, explitcit,year, danceability</li>
    </ul>
</ul>

<br>

**Insights** 
<br>
**Answering key business questions**

1. Artist analysis  
2. Song analysis
3. Time analysis, would older generation prefer other type of music ?
4. Make my playlist - focus on the danceability feature of the music <br>
    a. For a young target <br>
    b. For a mature target 

Find the dataset @
https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks

# About the dataset
## Imported from kaggle 

<ul>
Primary:<br>
- id (Id of track generated by Spotify) <br><br>
Numerical: <br>
- acousticness (Ranges from 0 to 1) <br>
- danceability (Ranges from 0 to 1)<br>
- energy (Ranges from 0 to 1)<br>
- duration_ms (Integer typically ranging from 200k to 300k)<br>
- instrumentalness (Ranges from 0 to 1)<br>
- valence (Ranges from 0 to 1)<br>
- popularity (Ranges from 0 to 100)<br>
- tempo (Float typically ranging from 50 to 150)<br>
- liveness (Ranges from 0 to 1)<br>
- loudness (Float typically ranging from -60 to 0)<br>
- speechiness (Ranges from 0 to 1)<br>
- year (Ranges from 1921 to 2020) <br><br>
Dummy:<br>
- mode (0 = Minor, 1 = Major)<br>
- explicit (0 = No explicit content, 1 = Explicit content) <br><br>
Categorical:<br>
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)<br>
- artists (List of artists mentioned)<br>
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)<br>
- name (Name of the song)
 </ul>

In [1]:
#Importing APIs
import pandas as pd
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window
from IPython.display import display, Markdown
from pyspark.sql import SQLContext


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sqlContext = SQLContext(spark)

In [2]:
#defining dataset's schema 
spotifySchema = StructType(\
    [StructField("acousticness",DoubleType(),True),\
     StructField("artists",StringType(),True),\
     StructField("danceability",DoubleType(),True),\
     StructField("duration_ms",IntegerType(),True),\
     StructField("energy",DoubleType(),True),\
     StructField("explicit",IntegerType(),True),\
     StructField("id",IntegerType(),True),\
     StructField("instrumentalness",DoubleType(),True),\
     StructField("key",IntegerType(),True),\
     StructField("liveness",IntegerType(),True),
    StructField("year",IntegerType(),True)])

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">    Data set metadata analysis </div>

In [3]:
#Load dataset
spotify_with_schema = spark.read.load("data.csv",
                     format="csv", sep=",", Schema=spotifySchema, header="true", mode = "PERMISSIVE")

print(spotify_with_schema.show(3))
spotify_with_schema.select(count("id").alias("Total_songs")).show()

#sampling the dataset
sample_spotify = spotify_with_schema.sample(fraction = 0.7, seed=1234)
sample_spotify.select(count("id").alias("Total__sample_songs")).show()
sample_spotify.printSchema()

+------------+--------------------+------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+--------------------+----------+------------+-----------+-------+------------------+----+
|acousticness|             artists|danceability|duration_ms|             energy|explicit|                  id|instrumentalness|key|liveness|           loudness|mode|                name|popularity|release_date|speechiness|  tempo|           valence|year|
+------------+--------------------+------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+--------------------+----------+------------+-----------+-------+------------------+----+
|       0.991|     ['Mamie Smith']|       0.598|     168333|0.22399999999999998|       0|0cS0A1fUEUd1EW3Fc...|        0.000522|  5|   0.379|            -12.628|   0|Keep A Song In Yo...|        12|        1920|     0.0936|149.976|     

In [4]:
#Summary of the data using panda 
spotifypd = sample_spotify.toPandas()
print(spotifypd.head(3))
print(spotifypd.describe())


  acousticness                            artists danceability duration_ms  \
0        0.993                    ['Mamie Smith']        0.647      163827   
1     0.000173                ['Oscar Velazquez']         0.73      422087   
2        0.996  ['Mamie Smith & Her Jazz Hounds']        0.424      198627   

                energy explicit                      id    instrumentalness  \
0  0.18600000000000005        0  11m7laMUgmOKqI3oYzuhne            1.76e-05   
1   0.7979999999999999        0  19Lc5SfJJ5O1oaxY0fpwfh  0.8009999999999999   
2                0.245        0  3HnrHGLE9u2MjHtdobfWl9               0.799   

  key liveness             loudness mode  \
0   0    0.519  -12.097999999999999    1   
1   2    0.128               -7.311    1   
2   5    0.235               -11.47    1   

                                                name popularity release_date  \
0                                       Golfing Papa          4         1920   
1  True House Music - Xavier Sant

In [5]:
#null values 
{col:sample_spotify.filter(sample_spotify[col].isNull()).count() for col in sample_spotify.columns}

{'acousticness': 0,
 'artists': 0,
 'danceability': 0,
 'duration_ms': 0,
 'energy': 0,
 'explicit': 0,
 'id': 0,
 'instrumentalness': 0,
 'key': 0,
 'liveness': 0,
 'loudness': 0,
 'mode': 0,
 'name': 0,
 'popularity': 0,
 'release_date': 0,
 'speechiness': 0,
 'tempo': 0,
 'valence': 0,
 'year': 0}

In [6]:
#average of columns for songs after 2000s
average = [sample_spotify.filter(sample_spotify['year'] > 2000  )
 .agg(avg(col(coll)).alias(coll)).show() 
 for coll in sample_spotify.columns]


+------------------+
|      acousticness|
+------------------+
|0.2421711288802191|
+------------------+

+-------+
|artists|
+-------+
|   null|
+-------+

+------------------+
|      danceability|
+------------------+
|0.5843683659314745|
+------------------+

+------------------+
|       duration_ms|
+------------------+
|246132.19902205575|
+------------------+

+------------------+
|            energy|
+------------------+
|0.6589337063427637|
+------------------+

+------------------+
|          explicit|
+------------------+
|0.1529685115827438|
+------------------+

+----+
|  id|
+----+
|null|
+----+

+-------------------+
|   instrumentalness|
+-------------------+
|0.20539563344430553|
+-------------------+

+-----------------+
|              key|
+-----------------+
|5.296435011790817|
+-----------------+

+------------------+
|          liveness|
+------------------+
|0.2140592062005829|
+------------------+

+------------------+
|          loudness|
+------------------+
|-

### C. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** Artists, songs
* **Metrics:**  duration, loudness, popularity, tempo,valence ...
* **Dimensions:** explicit, year, release date ...

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Artist analysis </div>

**What are the most 'famous' artist  ?**
<br> **Business questions**:
- Evaluate remuneration for artist 
- Recommendation for random playlist 

In [7]:


#sorted by artist that sang the most songs  
sample_spotify.groupby("artists")\
.agg(count("id").alias("songs"))\
.sort("songs",ascending = False)\
.show(15) 

#sorted by the most popular songs (average of all the artist songs popularity) /25 songs min
famous_artist = sample_spotify.groupby("artists")\
.agg(round(avg("popularity")).alias("average_popularity"), count("id").alias("songs"))\
.sort("average_popularity",ascending = False)\
.filter(col("songs" )> 25) 

famous_artist.show(15) 

+--------------------+-----+
|             artists|songs|
+--------------------+-----+
|['Tadeusz Dolega ...|  918|
|['Эрнест Хемингуэй']|  850|
|['Эрих Мария Рема...|  739|
|['Francisco Canaro']|  629|
| ['Ignacio Corsini']|  439|
|   ['Frank Sinatra']|  397|
|   ['Elvis Presley']|  334|
|       ['Bob Dylan']|  325|
|['Francisco Canar...|  325|
|  ['The Beach Boys']|  322|
|   ['Fleetwood Mac']|  317|
|['The Rolling Sto...|  304|
|     ['Johnny Cash']|  303|
|     ['Miles Davis']|  300|
|     ['The Beatles']|  289|
+--------------------+-----+
only showing top 15 rows

+--------------------+------------------+-----+
|             artists|average_popularity|songs|
+--------------------+------------------+-----+
|      ['Juice WRLD']|              75.0|   27|
|   ['Ariana Grande']|              72.0|   43|
|     ['Post Malone']|              71.0|   26|
|             ['BTS']|              71.0|   41|
|      ['Ed Sheeran']|              69.0|   28|
|   ['One Direction']|              67.

->  The more songs does not always mean the merrier as seen in the second table. <br>
-> However one can noticed that some famous older artist are not noted popular while they played a lot of songs
   artists such as Bob Dylan, Elvis Presley,the beattles, the beachboys, to evaluate in the time serie analysis

**Is an artist not recommended for chidren ?** <br>
Most explicit artist with at least 10 recorded songs 

In [8]:
sample_spotify.groupby("artists")\
.agg(avg("explicit").alias("average_content"), count("id").alias("songs"))\
.sort("average_content",ascending = False)\
.withColumn("average_content", when(col("average_content") > 1, 1)
                                .otherwise(col("average_content")))\
.filter(col("songs" )> 10) \
.show(15) 

+--------------------+---------------+-----+
|             artists|average_content|songs|
+--------------------+---------------+-----+
|   "[""Anita O'Day""|            1.0|   17|
|           ['Logic']|            1.0|   14|
|['Todd Glass', 'B...|            1.0|   31|
|         ['DJ Quik']|            1.0|   15|
|        ['Agonoize']|            1.0|   39|
|        ['TON Jayy']|            1.0|   20|
|          ['K-Rino']|            1.0|   13|
|['South Park Mexi...|            1.0|   23|
|    ['Trippie Redd']|            1.0|   14|
|  ['The Skinflicks']|            1.0|   11|
|  ['Kendrick Lamar']|            1.0|   28|
|       ['Geto Boys']|            1.0|   14|
|['Эрнест Хемингуэй']|            1.0|  850|
|       ['Mobb Deep']|            1.0|   14|
|['Эрих Мария Рема...|            1.0|  739|
+--------------------+---------------+-----+
only showing top 15 rows



**What are the artist to dance to ?** <br>
Artist with 10 songs minimum

In [9]:

dance = sample_spotify.groupby("artists")\
.agg((round(avg("danceability"),2)*100).alias("average_dance_prc"), 
     count("id").alias("songs"),
     avg("explicit").alias("artists_explicit_avg"),
    avg("popularity").alias("artists_popularity_avg"))\
.sort("average_dance_prc",ascending = False)\
.filter(col("songs" )> 10)
dance.show(15) 

+--------------------+-----------------+-----+--------------------+----------------------+
|             artists|average_dance_prc|songs|artists_explicit_avg|artists_popularity_avg|
+--------------------+-----------------+-----+--------------------+----------------------+
|       ['Too $hort']|             88.0|   19|   0.631578947368421|     39.21052631578947|
|   ['Sir Mix-A-Lot']|             88.0|   11|  0.8181818181818182|     38.09090909090909|
|     ['2 LIVE CREW']|             88.0|   16|              0.9375|               35.3125|
|      ['Run–D.M.C.']|             86.0|   21|                 0.0|     37.38095238095238|
|['Mi Banda El Mex...|             85.0|   14|                 0.0|    45.785714285714285|
|  ['Gregory Isaacs']|             85.0|   18|                 0.0|    37.166666666666664|
|         ['DJ Quik']|             84.0|   15|                 1.0|    37.266666666666666|
|   ['The Kiboomers']|             83.0|   11|                 0.0|     33.27272727272727|

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Song analysis </div>

**Most sang names**

In [10]:
sample_spotify.filter((col("name")!= "1") & (col("name") != "0")) \
.groupby("name") \
.agg(count("id").alias("songs"), 
     min("year").alias("Realeased_year"),
    max("year").alias("Last_sang"))\
.sort("songs",ascending = False)\
.show(15) 

+--------------------+-----+--------------+---------+
|                name|songs|Realeased_year|Last_sang|
+--------------------+-----+--------------+---------+
|     White Christmas|   77|          1936|     2009|
|   Winter Wonderland|   61|          1932|     2018|
|        Silent Night|   59|          1943|     2012|
|        Jingle Bells|   56|          1932|     2011|
|          2000 Years|   49|          1987|     2021|
|         Sleigh Ride|   44|          1932|     2015|
|          Summertime|   37|          1933|     2020|
|      Happy New Year|   37|          1957|     2021|
|       99 Year Blues|   35|          1972|     2012|
|  The Christmas Song|   34|          1943|     2012|
|            Overture|   32|          1921|     2004|
|        O Holy Night|   29|          1945|     1999|
|        Silver Bells|   28|          1945|     2011|
|My Only Wish (Thi...|   27|          2000|     2019|
|           Ave Maria|   27|          1926|     2015|
+--------------------+-----+

It seems christmas songs are old classic !

**What are the song to dance to ?** <br>
Song with 75% popularity minimum

In [11]:
song_dance =sample_spotify.filter(col("popularity" )> 0.75) \
.groupby("name")\
.agg((round(max("danceability"),2)*100).alias("max_dance_prc"), 
     count("artists").alias("number of retake"),
     max("artists").alias("last_artist"),
     max("explicit").alias("song_explicit"))\
.sort("max_dance_prc",ascending = False)
song_dance.show(15) 

+--------------------+-------------+----------------+--------------------+-------------+
|                name|max_dance_prc|number of retake|         last_artist|song_explicit|
+--------------------+-------------+----------------+--------------------+-------------+
|Funky Cold Medina...|         99.0|               1|        ['Tone-Loc']|            0|
|             Go Girl|         99.0|               1|['Pitbull', 'Trin...|            1|
|   Funky Cold Medina|         99.0|               1|        ['Tone-Loc']|            0|
|  I Need You Tonight|         98.0|               3|  ['Punkin Machine']|            1|
|      Black Is Black|         98.0|               2|      ['Los Bravos']|            0|
|       Give It To Me|         98.0|               2|['Timbaland', 'Ju...|            1|
|   Barney Theme Song|         98.0|               1|          ['Barney']|            0|
|            Teachers|         98.0|               1|       ['Daft Punk']|            0|
|Ice Ice Baby - Ra...

Usin artist with 10 songs minimum:

In [12]:
#inner join on dance artists
Combined = dance.join(song_dance,dance.artists == song_dance.last_artist)
Combined = Combined.drop("last_artist").drop("number of retake")
Combined.sort("max_dance_prc",ascending = False).show(15)

+--------------------+-----------------+-----+--------------------+----------------------+--------------------+-------------+-------------+
|             artists|average_dance_prc|songs|artists_explicit_avg|artists_popularity_avg|                name|max_dance_prc|song_explicit|
+--------------------+-----------------+-----+--------------------+----------------------+--------------------+-------------+-------------+
|       ['Daft Punk']|             71.0|   31|                 0.0|    53.903225806451616|            Teachers|         98.0|            0|
|   ['Sir Mix-A-Lot']|             88.0|   11|  0.8181818181818182|     38.09090909090909|             Beepers|         98.0|            0|
|    ['Selena Gomez']|             69.0|   12|                 0.0|     64.41666666666667|            Bad Liar|         97.0|            0|
|      ['Run–D.M.C.']|             86.0|   21|                 0.0|     37.38095238095238|         It's Tricky|         97.0|            0|
|            ['UB40'

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Time series </div>

In [13]:
#yearly analysis
sample_spotify = sample_spotify.withColumn("year", sample_spotify["year"].cast("integer"))\
.filter(col("year" )> 1900)

sample_spotify.filter((col("explicit") <= 1) & (col('year') != 1) & (col('year') != 0))\
.groupby(col("year"))\
.agg((round(avg("danceability"),1)*100).alias("average_dance_prc"), 
count("id").alias("Songs"),
(round(avg("explicit"),2)*100).alias("average_explicit_prc"),
(round(avg("popularity"),1)).alias("average_popularity_prc"),
(round(stddev("popularity"),1)).alias("std_popularity"))\
.sort(col("year"),ascending = False)\
.show(60) 



+----+-----------------+-----+--------------------+----------------------+--------------+
|year|average_dance_prc|Songs|average_explicit_prc|average_popularity_prc|std_popularity|
+----+-----------------+-----+--------------------+----------------------+--------------+
|2021|             70.0| 1284|                10.0|                   7.6|          18.5|
|2020|             60.0| 3025|                17.0|                  24.1|          30.1|
|2019|             60.0| 1637|                23.0|                  32.1|          32.3|
|2018|             60.0| 1924|                23.0|                  27.0|          32.0|
|2017|             60.0| 1528|                23.0|                  32.0|          32.3|
|2016|             60.0| 1660|                18.0|                  29.9|          30.4|
|2015|             60.0| 1616|                15.0|                  29.8|          30.3|
|2014|             60.0| 1552|                16.0|                  29.1|          29.3|
|2013|    

- Clear increasing trend in the dancability of songs over the years 
- Explicit content started in 1994 and seemed to over around an average as deribed in post 2000 songs analysis: 15%
- To my suprise no major trend in the average number of songs nor popularity
- Looking closer the deviation of the popularity is larger in the recent years (since 2004 where the deviation reached 18.4 never going lower in the later years) which suggest a normal distribution of popularity with few very popular artist at the top 

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Playlist </div>

**Process**
1. Generates number of desired users 
2. Assign a sample of music with desired attributes (the focus here is danceability) to the users 
3. Create a dataframe with the desired number of songs predicted for each users 
4. search in Spotify catalog and assign the songs to each users 
<br><br>
**Business question**
<br><br>
<div style= "text-align:center"> The right music recommendation provides multiple gains from generating new subscription, reducing customer turnover to selling packaged playlist to restaurants, bar..</div>



In [14]:
#generate user based on random sample of songs woth a rating of dancability above 0.5, artist with at least 10 songs 
#from the combined dataset generated earlier 
#generate 10 users with each a list of 80-120 songs
#we will use those user throughout the playlist comopsition 
#I will only take a small sample of the original datasets for faster result here
sampleplaylist = Combined.sample(fraction = 0.3)
user_count = 10
users = sampleplaylist.sample(fraction = 0.007, seed=1234).withColumn("ID", lit(0)).toPandas()



In [15]:
#last 5 songs of the first user
print(users.tail(5))
#Number of songs of users 
users.groupby("ID").count()

                 artists  average_dance_prc  songs  artists_explicit_avg  \
103        ['Ry Cooder']               56.0     12              0.000000   
104  ['Ella Fitzgerald']               41.0    263              0.000000   
105      ['Stan Rogers']               57.0     12              0.000000   
106               ['白光']               41.0     16              0.000000   
107      ['David Bowie']               49.0    213              0.004695   

     artists_popularity_avg                                name  \
103               36.083333                     Available Space   
104               22.057252  Let's Take A Walk Around The Block   
105               33.250000            Rolling Down to Old Maui   
106                1.812500                               東山一把情   
107               35.159624           Amsterdam - 2015 Remaster   

     max_dance_prc song_explicit  ID  
103           57.0             0   0  
104           52.0             0   0  
105           57.0     

,artists,average_dance_prc,songs,artists_explicit_avg,artists_popularity_avg,name,max_dance_prc,song_explicit
ID,,,,,,,,
0,108,108,108,108,108,108,108,108


In [16]:
for i in range(1,user_count):
    rand_sample = sampleplaylist.sample(fraction = 0.007).withColumn("ID", lit(i)).toPandas()
    users = users.append(rand_sample,ignore_index=True) 

In [17]:
#last 5 songs of the last (10th) user
users.tail(5)

,artists,average_dance_prc,songs,artists_explicit_avg,artists_popularity_avg,name,max_dance_prc,song_explicit,ID
953,['Judas Priest'],38.0,92,0.000,36.000000,Fire Burns Below,29.0,0,9
954,"['Louis Prima', 'Keely Smith']",63.0,11,0.000,14.363636,Luigi,73.0,0,9
955,['A Perfect Circle'],51.0,16,0.125,53.187500,Orestes,46.0,0,9
956,['Bob Dylan'],51.0,325,0.000,29.544615,Most Likely You Go Your Way (And I'll Go Mine)...,34.0,0,9
957,['Elvis Presley'],50.0,334,0.000,34.416168,Welcome to My World - Live at The Honolulu Int...,21.0,0,9


The best at this point would be to desing an item based neighborhood recommendation system, for the time being I will focus on a simpler model (that could be combined with a recommendation system later ?)

For the sake of this assingment we will focus on:
- Song not yet listened to
- artists danceability over or equal the average danceability of artist listened to 
- Songs with at list with danceability higher than the minimum danceability of songs listened to

In [18]:
#generate a pyspark user dataframe
spotifySchema2 = StructType(\
    [
     StructField("artists_u",StringType(),True),\
     StructField("avgdanceability_u",DoubleType(),True),\
     StructField("songs_u",IntegerType(),True),\
     StructField("artists_explicit_avg_u",DoubleType(),True),\
    StructField("artists_popularity_avg_u",DoubleType(),True),\
     StructField("name",StringType(),True),\
     StructField("Songdanceability_u",DoubleType(),True),\
    StructField("song_explicit_u",StringType(),True),\
    StructField("ID_u",IntegerType(),True)])
usersdf = spark.createDataFrame(data = users,schema = spotifySchema2)
usersdf = usersdf.drop("artists_explicit_avg_u").drop("song_explicit_u").drop("artists_popularity_avg_u")

In [19]:
#total of generated users songs 
usersdf.show(10)

+--------------------+-----------------+-------+--------------------+------------------+----+
|           artists_u|avgdanceability_u|songs_u|                name|Songdanceability_u|ID_u|
+--------------------+-----------------+-------+--------------------+------------------+----+
|       ['Metallica']|             37.0|    155|Nothing Else Matt...|28.000000000000004|   0|
|['Armin van Buuren']|             50.0|    152|A State Of Trance...|              80.0|   0|
|     ['The Beatles']|             52.0|    289|Let It Be - Remas...|              45.0|   0|
|['Joan Jett & The...|             52.0|     16|        Fake Friends|              49.0|   0|
|    ['Ricky Skaggs']|             59.0|     15|Don't Get Above Y...|              63.0|   0|
|['Billy Currington']|             64.0|     11|    People Are Crazy|              65.0|   0|
|    ['Frankie Ruiz']|             72.0|     24|          Y No Puedo|              61.0|   0|
|  ['Linda Ronstadt']|             49.0|     61|Los Laureles

In [20]:
#join the users song history to the full repertory of songs 
#generate a static value - 1000 - to songs not yet listenned 
Complete = sampleplaylist.join(usersdf,on = "name",how="left").withColumn("ID_u",when(col("ID_u").isNull(), 1000)
                                                                   .otherwise(col("ID_u")))


In [21]:
#recommended songs (10) for each users 
#step 1 create a dataframe witht he id of the desired number of predicted songs 
Complete.registerTempTable("compsql");
songs_top_predict = 10
def create_song(songs_top_predict = 10):
    row_in = []
    for i in range(0,songs_top_predict):
        row_in.append(i)
    joined_reco = spark.createDataFrame(row_in, IntegerType()).withColumnRenamed("value","id")
    return joined_reco
joined_reco = create_song()
joined_reco.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



**Playlist without adaptation given the age of the user**

In [22]:
#step 2 predict the songs given the limit of the desired number of songs repreateadly for the total number of users to predict
#the query is randomly ordered to produce different playlist for each users 
#since I needed nested select statement which are not possible in Pyspark I have used an SQL query 
for i in range(0,user_count):
    ide = i
    user = ("user" + str(ide))
    ana = spark.sql("SELECT name AS "+ user +", artists AS "+ user +" \
                        FROM compsql  WHERE ID_u != "+ str(ide) +" AND \
                        average_dance_prc > (SELECT mean(avgdanceability_u) FROM compsql WHERE ID_u = "+ str(ide) +") AND \
                        max_dance_prc > (SELECT min(Songdanceability_u) FROM compsql WHERE ID_u = "+ str(ide) +") \
                        ORDER BY RAND() \
                        LIMIT "+str(songs_top_predict)+"")
    ana = ana.withColumn("id", monotonically_increasing_id())
    joined_reco = joined_reco.join(ana, "id")
recopd = joined_reco.toPandas()
recopd.head

<bound method NDFrame.head of Empty DataFrame
Columns: [id, user0, user0, user1, user1, user2, user2, user3, user3, user4, user4, user5, user5, user6, user6, user7, user7, user8, user8, user9, user9]
Index: []

[0 rows x 21 columns]>

**Playlist for younger Generation**

Keep the previous requirements adding:
- Ban artist with regular explicit content 
- Ban songs with explicit content 
- Select based on a popularity 

In [23]:
#predict 20 songs
young_tble = create_song(20)
songs_top_predict = 20
#Predict 20 songs for 1 user
user_count = 1   
for i in range(0,user_count):
    ide = i
    user = ("user" + str(ide))
    ana2 = spark.sql("SELECT name AS "+ user +", artists AS "+ user +" \
                        FROM compsql  WHERE ID_u != "+ str(ide) +" AND artists_popularity_avg > 0.5 AND \
                        song_explicit = '0' AND \
                        artists_explicit_avg < 0.5 AND \
                        average_dance_prc > (SELECT mean(avgdanceability_u) FROM compsql WHERE ID_u = "+ str(ide) +") AND \
                        max_dance_prc > (SELECT min(Songdanceability_u) FROM compsql WHERE ID_u = "+ str(ide) +")  \
                        ORDER BY RAND() \
                        LIMIT "+str(songs_top_predict)+"")
    ana2 = ana2.withColumn("id", monotonically_increasing_id())
    young_tble = young_tble.join(ana2, "id")
young_tble = young_tble.toPandas()
young_tble.head

<bound method NDFrame.head of     id                                              user0  \
0    0               The Seventh Stranger - 2010 Remaster   
1    1                                          Housework   
2    2  The Cowboy In The Continental Suit - Single Ve...   
3    3                                A Mentir A La Calle   
4    4                                     El Amor Soñado   
5    5                                                Ego   
6    6                                        Mississippi   
7    7                                   Oh Why? - Take 9   
8    8                    Black Magic Woman - 2018 Master   
9    9                                      No More Dream   
10  10                                Cachito - Unplugged   
11  11                                        I'm No Good   
12  12                                          We Danced   
13  13                                          You And I   
14  14                               Free - 2002 Remast

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Advice famous album   </div>

**Business question** 
1. How to propose the most appreciated album as a retailer ?
2. How to advice playlist based on famous albums ? 

In [24]:
#Create a limited datasets with the most famous artist with feature used int he most famous artist analysis further up 
#that is using the number of songs and popularity -- the limited dataset might truncate that number of songs unadvertively
famous = Combined.filter((col("artists_popularity_avg") > 0.9) & (col("songs" )> 25) )\
.select(col("artists"),"songs","artists_popularity_avg","name").sort("artists_popularity_avg",ascending = False).limit(300)
famous.show(40)

+-----------------+-----+----------------------+--------------------+
|          artists|songs|artists_popularity_avg|                name|
+-----------------+-----+----------------------+--------------------+
|   ['Juice WRLD']|   27|     75.29629629629629| Armed And Dangerous|
|   ['Juice WRLD']|   27|     75.29629629629629|          Bad Energy|
|   ['Juice WRLD']|   27|     75.29629629629629|        Lucid Dreams|
|   ['Juice WRLD']|   27|     75.29629629629629|             Candles|
|   ['Juice WRLD']|   27|     75.29629629629629|             Robbery|
|   ['Juice WRLD']|   27|     75.29629629629629|        I'll Be Fine|
|   ['Juice WRLD']|   27|     75.29629629629629|               Rider|
|   ['Juice WRLD']|   27|     75.29629629629629|           Can't Die|
|   ['Juice WRLD']|   27|     75.29629629629629|      Rich And Blind|
|   ['Juice WRLD']|   27|     75.29629629629629|         Lean Wit Me|
|   ['Juice WRLD']|   27|     75.29629629629629|         Screw Juice|
|   ['Juice WRLD']| 

In [25]:
#multiple creation of dataframe list to pivot the artist column: 
#An iterative id per song per artist was needed to proper pivot the artist variable and get all his songs as a result 
groups = [x[0] for x in famous.select("artists").distinct().collect()]
groups_list = [famous.filter(col('artists')==x) for x in groups]
group_ids = [x.withColumn("id", monotonically_increasing_id()) for x in groups_list]
group_pivoted = [x.groupby("id").pivot("artists").agg(max("name")) for x in group_ids]
[x.show() for x in group_pivoted]

+---+--------------------+
| id|      ['Juice WRLD']|
+---+--------------------+
|  0| Armed And Dangerous|
|  1|          Bad Energy|
|  2|        Lucid Dreams|
|  3|             Candles|
|  4|             Robbery|
|  5|        I'll Be Fine|
|  6|               Rider|
|  7|           Can't Die|
|  8|      Rich And Blind|
|  9|         Lean Wit Me|
| 10|         Screw Juice|
| 11|           HeMotions|
| 12|          Out My Way|
| 13|      Scared Of Love|
| 14|      Flaws And Sins|
| 15|             Legends|
| 16|All Girls Are The...|
| 17|     Hear Me Calling|
| 18|           Stay High|
| 19|        Make It Back|
+---+--------------------+
only showing top 20 rows

+---+--------------------+
| id|   ['Ariana Grande']|
+---+--------------------+
|  0|       Santa Tell Me|
|  1|             7 rings|
|  2|           positions|
|  3|      Tattooed Heart|
|  4|       thank u, next|
|  5|              Baby I|
|  6|       get well soon|
|  7|Wit It This Chris...|
|  8|               nasty|
| 

[None, None, None, None, None, None, None, None, None, None, None]

<div style="color:#1DB954;  font-size: 30px; text-align:center;  font-weight: bold; background-color:black; padding:12px">     Predict popularity   </div>

In [26]:
from pyspark.ml.feature import StringIndexer
sample_spotify = sample_spotify.dropna(how='any')
regression_df = sample_spotify.withColumn("popularity", when(col("popularity") >= 0.5, 1.0)
                                .otherwise(0.0))


In [27]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
required_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
                     'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

for colu in required_features:
    print(colu)
    cole = colu + "i"
    regression_df = regression_df.withColumn(colu,regression_df[colu].cast("float"))
    regression_df = regression_df.withColumn(colu,round(regression_df[colu],2))


regression_df = regression_df.select('popularity','acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
                     'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence')

acousticness
danceability
duration_ms
energy
explicit
instrumentalness
liveness
loudness
speechiness
tempo
valence


In [28]:
 assembler = VectorAssembler(inputCols=required_features,\
                            outputCol='features',handleInvalid='skip')
    
regressionfeatureddf = assembler.transform(regression_df)

regressionfeatureddf.select("features").show()
regressionfeatureddf.show()

+--------------------+
|            features|
+--------------------+
|[0.99000000953674...|
|[0.0,0.7300000190...|
|[1.0,0.4199999868...|
|[0.99000000953674...|
|[1.0,0.4699999988...|
|[1.0,0.4699999988...|
|[0.00999999977648...|
|[0.94999998807907...|
|[1.0,0.5799999833...|
|[0.99000000953674...|
|[0.0,0.6399999856...|
|[0.99000000953674...|
|[0.98000001907348...|
|[0.97000002861022...|
|[1.0,0.4699999988...|
|[0.75,0.529999971...|
|[0.99000000953674...|
|[0.01999999955296...|
|[0.07999999821186...|
|[0.00999999977648...|
+--------------------+
only showing top 20 rows

+----------+------------+------------+-----------+------+--------+----------------+--------+--------+-----------+------+-------+--------------------+
|popularity|acousticness|danceability|duration_ms|energy|explicit|instrumentalness|liveness|loudness|speechiness| tempo|valence|            features|
+----------+------------+------------+-----------+------+--------+----------------+--------+--------+-----------+------+--

In [29]:
from pyspark.ml.classification import RandomForestClassifier


(trainingDF, testDF) = regressionfeatureddf.randomSplit([0.7,0.3])

rfcAlgorithm = RandomForestClassifier(labelCol='popularity',\
                                      featuresCol= 'features',\
                                      maxDepth=5)

#3. Train the algorithm to build the model and apply it on the test data set.
model = rfcAlgorithm.fit(trainingDF)
predictions = model.transform(testDF)
predictions.dtypes

[('popularity', 'double'),
 ('acousticness', 'float'),
 ('danceability', 'float'),
 ('duration_ms', 'float'),
 ('energy', 'float'),
 ('explicit', 'float'),
 ('instrumentalness', 'float'),
 ('liveness', 'float'),
 ('loudness', 'float'),
 ('speechiness', 'float'),
 ('tempo', 'float'),
 ('valence', 'float'),
 ('features', 'vector'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='popularity',\
                                              predictionCol='prediction',\
                                              metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print('The accuracy of the model is ', accuracy)

The accuracy of the model is  0.8367858017802215
